# Edward's Bowl of Spaghetti Code

## Imports and DataFrame Initialization

In [2]:
# TODO KETE SETSUNASA NI WA
import pandas as pd
# TODO have input prompt instead of hard-coding
df_OLD = pd.read_excel("files/monthly_production_05-2019.xlsx")
df_NEW = pd.read_excel("files/mothpro2019.xlsx")

#dfRevTest = pd.read_excel('files/monthly_revenue_04-2019.xlsx').fillna(0)
dfCYPro = pd.read_excel('files/federal_production_CY08-17.xlsx')

## For Setting Header Format

In [11]:
# Reads Header Config File
def read_hconfig():
    columns = []
    with open('config/headerdef.txt') as hdef:
        # Seperated by commas, no whitespace plz :(
        columns = hdef.readline().split(',')
    return columns

# Returns Header List based on Excel file
# Use if large amount of Field Names
def set_header(samplefile):
    return list(samplefile.columns)


header = set_header(df_OLD)
print(header)

['Month', 'Calendar Year', 'Land Class', 'Land Category', 'Commodity', 'Volume']


## For Setting Unit Dictionary

In [19]:
# Reads Unit Config File
# Commodity and Unit seperated by an equals sign " = "
# Fast, Best for small number of Units / after initial read
def read_uconfig():
    units = {}
    with open('config/unitdef.txt') as udef:
        for line in udef:
            split = line.split(" = ")
            add_item(split[0], split[1].strip(), units)
    return units


# Returns Unit Dictionary on Excel file
# Very Slow, but better than manually setting large # of units
def make_unit_dict(file):
    units = {}
    for row in file['Product']:
        line = split_unit(row)
        k,v = line[0], line[1]
        add_item(k,v,units)
    return units


# Returns Split String based on Commodity and Unit
def split_unit(string):
    # For general purpose commodities
    if "(" in string:
        split = string.rsplit(" (",1)
        split[1] = split[1].rstrip(")")
        return split
    # The comma is for Geothermal
    elif "," in string:
        return string.split(", ",1)
    # In case no unit is found
    return [string,'']


# Adds Key with a Set of Values
def add_item(key, value, dictionary):
    if key in dictionary:
        dictionary[key].add(value)
    else:
        dictionary[key] = {value}

        
# Wrties to config to speed up process later
# Maybe have it write seperate files for each type?
def write_units(units):
    config = open("config/unitdef.txt","w")
    for k,v in units.items():
        for u in v:
            line = k + ' = ' +  u.strip("'")  + '\n'
            config.write(line)
        
        
write_units(make_unit_dict(dfCYPro))
units = read_uconfig()
units

{'Gas': {'mcf'},
 'Oil': {'bbl'},
 'Coal': {'tons'},
 'Wavellite': {'spcmn'},
 'Quartz Crystal': {'tickets/pounds'},
 'Salt': {'tons'},
 'Geothermal - Electrical Generation': {'Kilowatt Hours',
  'Other',
  'Thousands of Pounds'},
 'Geothermal - Direct Utilization': {'Hundreds of Gallons',
  'Millions of BTUs'},
 'Anhydrous Sodium Sulfate': {'tons'},
 'Borax-Decahydrate': {'tons'},
 'Borax-Pentahydrate': {'tons'},
 'Boric Acid': {'tons'},
 'Soda Ash': {'tons'},
 'Sodium Bi-Carbonate': {'tons'},
 'Phosphate Raw Ore': {'tons'},
 'Gold': {'oz'},
 'Copper Concentrate': {'tons'},
 'Lead Concentrate': {'tons'},
 'Zinc Concentrate': {'tons'},
 'Leonardite': {'tons'},
 'Langbeinite': {'tons'},
 'Muriate Of Potash-Granular': {'tons'},
 'Muriate Of Potash-Standard': {'tons'},
 'Potash': {'tons'},
 'Sylvite-Raw Ore': {'tons'},
 'Brine Barrels': {'bbl'},
 'Clay': {'tons'},
 'Magnesium Chloride Brine': {'tons'},
 'Manure Salts': {'tons'},
 'Gilsonite': {'tons'},
 'Limestonse': {'tons'},
 'Borate Pr

## Check Header Function

In [20]:
def check_header(file):
    columns = file.columns
    uncheckedCols = set(columns)
    
    for i in range(len(header)):
        # Checks if Field in file and in correct column
        if columns.contains(header[i]):
            if columns[i] == header[i]:
                print(header[i] + ": True")
            else:
                print(header[i] + ": Wrong order")
            uncheckedCols.remove(header[i])
        else:
            # Field not present in the file
            print(header[i] + ': N/A')
    # Prints all fields not in the format
    print('\nNew Cols:', uncheckedCols)
        
check_header(dfCYPro)

Month: N/A
Calendar Year: Wrong order
Land Class: N/A
Land Category: N/A
Commodity: N/A
Volume: N/A

New Cols: {'Onshore/Offshore', 'Production Volume', 'Product', 'State/Offshore Region', 'CPS/Planning Area', 'FIPS Code', 'Category'}


## Two  Potential Unit Approaches
Compare File Dictionary
* Slightly Harder to implement
* More modular
* Better for documentation purposes

OR Compare File to Set
* Easier to implement
* Not as flexible

In [35]:
def make_unit_set(file):
    units = set()
    for u in file['Commodity']:
        units.add(u)
    return units


def check_unit_set(file, default):
    index = 0
    for u in file['Commodity']:
        if u not in default:
            print(index,u,'?')
        index+=1


def check_unit_dict(file, default):
    index = 0
    bad = False
    for u in file['Product']:
        line = split_unit(u)
        if default.__contains__(line[0]):
            if line[1] not in default.get(line[0]):
                print('Row ' + str(index) + ': Unknown Unit ' + line[1]  + ' For Item: ' + line[0])
                bad = True
        else:
            print('Row ' + str(index) + ': Unknown Item: ' + line[0])
            bad = True
        index+=1
    if bad != True:
        print('No Errors Found :)')
            

write_units(make_unit_dict(dfCYPro))
default_dict = read_uconfig()
check_unit_dict(dfCYPro,default_dict)

No Errors Found :)


In [20]:
df_OLD.loc[11,'Commodity']

'Gas Prod Vol (mcf)'